In [1]:
import asyncio
import hypersync
import nest_asyncio
import polars as pl

from hypersync import LogSelection, FieldSelection, LogField, TransactionField, BlockField

# expand polars df output
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

# to run asyncio loop in notebook
nest_asyncio.apply()

In [2]:
# Hypersync debugging
import os
import subprocess

# Set the RUST_LOG environment variable
os.environ['RUST_LOG'] = 'trace'

In [3]:
cd ../

/home/evan/Documents/mev_commit_sdk_py


/home/evan/Documents/mev_commit_sdk_py/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Query CommitmentProcessed (new)

In [4]:
async def get_commitment_processed():
    client = hypersync.HypersyncClient(
        hypersync.ClientConfig(url="https://mev-commit.hypersync.xyz"))

    block_height = await client.get_height()

    contract = "0x6856Eb630C79D491886E104D328834643B3F69E3".lower()  # oracle contract

    query = hypersync.Query(
        from_block=0,
        to_block=block_height,
        logs=[LogSelection(address=[contract])],
        field_selection=FieldSelection(
            log=[e.value for e in LogField],
            transaction=[e.value for e in TransactionField]
        )
    )

    config = hypersync.StreamConfig(
        hex_output=hypersync.HexOutput.PREFIXED,
        event_signature="CommitmentProcessed(bytes32 commitmentHash, bool isSlash)"
    )

    print("Running the query...")

    res = await client.get(query)

    print(
        f"Query returned {len(res.data.logs)} logs from contract {contract}")

    return await client.collect_arrow(query, config)

commitment_processed_data = asyncio.run(get_commitment_processed())

[2024-07-04T20:08:26Z DEBUG reqwest::connect] starting new connection: https://mev-commit.hypersync.xyz/


Running the query...
Query returned 20 logs from contract 0x6856eb630c79d491886e104d328834643b3f69e3


[2024-07-04T20:08:27Z TRACE hypersync_client::util] failed to decode body of a log, will write null instead. Error was: decode body tuple
    
    Caused by:
        0: buffer overrun while deserializing
        1: buffer overrun while deserializing
[2024-07-04T20:08:27Z TRACE hypersync_client::util] failed to decode body of a log, will write null instead. Error was: decode body tuple
    
    Caused by:
        0: buffer overrun while deserializing
        1: buffer overrun while deserializing
[2024-07-04T20:08:27Z TRACE hypersync_client::util] failed to decode body of a log, will write null instead. Error was: decode body tuple
    
    Caused by:
        0: buffer overrun while deserializing
        1: buffer overrun while deserializing
[2024-07-04T20:08:27Z TRACE hypersync_client::util] failed to decode body of a log, will write null instead. Error was: decode body tuple
    
    Caused by:
        0: buffer overrun while deserializing
        1: buffer overrun while deserializing


In [5]:
logs_df = pl.from_arrow(commitment_processed_data.data.logs)
decoded_logs_df = pl.from_arrow(commitment_processed_data.data.decoded_logs)

In [6]:
# hstack logs and decoded_logs_df
commitment_processed_df = logs_df.hstack(decoded_logs_df).filter(
    pl.col('commitmentHash').is_not_null())

In [7]:
# shows the `commitmentHash` and whether the provider was slashed or not
commitment_processed_df.head(5)

removed,log_index,transaction_index,transaction_hash,block_hash,block_number,address,data,topic0,topic1,topic2,topic3,commitmentHash,isSlash
bool,u64,u64,str,str,u64,str,str,str,str,str,str,str,bool
false,2,0,"""0xf87527dcc66a0fcc6ac982f32f6d678f03e92e2cf5afb2d277183f66e1f9b784""","""0xef50e3c95a91aa2949a53d897407759602765a56f20aad97c2b9e7f532016d94""",131372,"""0x6856eb630c79d491886e104d328834643b3f69e3""","""0xc9f730e8e2210c5cbea8c1fd8e57251b3ea60cf64be034b8acbe9645f6897d300000000000000000000000000000000000000000000000000000000000000001""","""0xddc1768a3a762a04e5fd3abea8ae3b60e23bcf290f4a032280e6a726611d41f5""",null,null,null,"""0xc9f730e8e2210c5cbea8c1fd8e57251b3ea60cf64be034b8acbe9645f6897d30""",true
false,1,0,"""0x344671414a2e6bb22edfd7ce6fc2b9642d1d16fcc1585df8278be67b3ebd7509""","""0xd4dc21f6aae0d847406c3e02975edcfb72541bb25217f60aae5269402489eeb6""",132689,"""0x6856eb630c79d491886e104d328834643b3f69e3""","""0xdb68a8ac279d872dc0ae9812bf370378e402756f54e848a0e09c7ab2f21d401a0000000000000000000000000000000000000000000000000000000000000000""","""0xddc1768a3a762a04e5fd3abea8ae3b60e23bcf290f4a032280e6a726611d41f5""",null,null,null,"""0xdb68a8ac279d872dc0ae9812bf370378e402756f54e848a0e09c7ab2f21d401a""",false
false,2,0,"""0xe05a6ad4850806f8a44828b7b9a7def33bdf661c41d4abfcc7c13580d7b600ba""","""0x9f01f140c73e66361c7a7444f185720c3040ba37cf5deeefa2ce6a8e31730733""",536069,"""0x6856eb630c79d491886e104d328834643b3f69e3""","""0x25d08081b4f88c61d4842ee9f6847cbe51f6a0eb171499e797c619fc8c1972540000000000000000000000000000000000000000000000000000000000000001""","""0xddc1768a3a762a04e5fd3abea8ae3b60e23bcf290f4a032280e6a726611d41f5""",null,null,null,"""0x25d08081b4f88c61d4842ee9f6847cbe51f6a0eb171499e797c619fc8c197254""",true
false,2,0,"""0xdbf283c093592d0114e3d57b2198f94baca461dbf43d7026ccfd2bbfad85e548""","""0x02b3968310adaf67ace73ce120a598f74a13a49b8c47f58e6706df66e9d9b292""",539070,"""0x6856eb630c79d491886e104d328834643b3f69e3""","""0x6cab531c0475314c37629e22f091438ba299d646cc9137720be45299d3323afb0000000000000000000000000000000000000000000000000000000000000001""","""0xddc1768a3a762a04e5fd3abea8ae3b60e23bcf290f4a032280e6a726611d41f5""",null,null,null,"""0x6cab531c0475314c37629e22f091438ba299d646cc9137720be45299d3323afb""",true
false,1,0,"""0xbdf08c56ec7f80e48c02a4b592375eed3fb7d7ca8aafa566ba518a3e8ddccfef""","""0x3edb125c96da536ae34107ba75adec2266e6432d1fdf9119649db294423991b9""",540572,"""0x6856eb630c79d491886e104d328834643b3f69e3""","""0xeaf0a73e8065541d017297b83d2e9723c6d4ef1c86e1b9f8ebe643e1e75eaf960000000000000000000000000000000000000000000000000000000000000000""","""0xddc1768a3a762a04e5fd3abea8ae3b60e23bcf290f4a032280e6a726611d41f5""",null,null,null,"""0xeaf0a73e8065541d017297b83d2e9723c6d4ef1c86e1b9f8ebe643e1e75eaf96""",false


In [8]:
commitment_processed_df.group_by('isSlash').agg(pl.len())

isSlash,len
bool,u32
false,6
true,10


### Query CommitmentStored

In [9]:
async def get_commitment_stored():
    client = hypersync.HypersyncClient(
        hypersync.ClientConfig(url="https://mev-commit.hypersync.xyz"))

    block_height = await client.get_height()

    # PreConfCommitmentStore contract
    contract = "0xCAC68D97a56b19204Dd3dbDC103CB24D47A825A3".lower()

    query = hypersync.Query(
        from_block=0,
        to_block=block_height,
        logs=[LogSelection(
            address=[contract],
            topics=[
                ["0xa4aab50afc443b845214b8f4e2e7c32ea42be39a84e532be779802c54ff8ffda"]],
        )],
        field_selection=FieldSelection(
            log=[e.value for e in LogField],
            transaction=[e.value for e in TransactionField]
        )
    )

    config = hypersync.StreamConfig(
        hex_output=hypersync.HexOutput.PREFIXED,
        event_signature="CommitmentStored(bytes32 indexed commitmentIndex, address bidder, address commiter, uint64 bid, uint64 blockNumber, bytes32 bidHash, uint64 decayStartTimeStamp, uint64 decayEndTimeStamp, string txnHash, bytes32 commitmentHash, bytes bidSignature, bytes commitmentSignature, uint64 dispatchTimestamp, bytes sharedSecretKey)",
    )

    print("Running the query...")

    res = await client.get(query)

    print(
        f"Query returned {len(res.data.logs)} logs from contract {contract}")

    return await client.collect_arrow(query, config)

commitment_stored_data = asyncio.run(get_commitment_stored())

[2024-07-04T20:08:28Z DEBUG reqwest::connect] starting new connection: https://mev-commit.hypersync.xyz/


Running the query...
Query returned 18 logs from contract 0xcac68d97a56b19204dd3dbdc103cb24d47a825a3


[2024-07-04T20:08:29Z TRACE hypersync_client::util] failed to decode body of a log, will write null instead. Error was: decode body tuple
    
    Caused by:
        0: buffer overrun while deserializing
        1: buffer overrun while deserializing
[2024-07-04T20:08:29Z TRACE hypersync_client::util] failed to decode body of a log, will write null instead. Error was: decode body tuple
    
    Caused by:
        0: buffer overrun while deserializing
        1: buffer overrun while deserializing
[2024-07-04T20:08:29Z TRACE hypersync_client::util] failed to decode body of a log, will write null instead. Error was: decode body tuple
    
    Caused by:
        0: buffer overrun while deserializing
        1: buffer overrun while deserializing
[2024-07-04T20:08:29Z TRACE hypersync_client::util] failed to decode body of a log, will write null instead. Error was: decode body tuple
    
    Caused by:
        0: buffer overrun while deserializing
        1: buffer overrun while deserializing


In [10]:
# ! Not correctly decoding the log fields, everything is null. Same code with cryo returns correct values.
commitment_stored_logs_df = pl.from_arrow(commitment_stored_data.data.logs)
commitment_stored_decoded_logs_df = pl.from_arrow(
    commitment_stored_data.data.decoded_logs)

In [11]:
# should not be null
commitment_stored_decoded_logs_df.head(5)

commitmentIndex,bidder,commiter,bid,blockNumber,bidHash,decayStartTimeStamp,decayEndTimeStamp,txnHash,commitmentHash,bidSignature,commitmentSignature,dispatchTimestamp,sharedSecretKey
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""0xc9f730e8e2210c5cbea8c1fd8e57251b3ea60cf64be034b8acbe9645f6897d30""",null,null,null,null,null,null,null,null,null,null,null,null,null
"""0xdb68a8ac279d872dc0ae9812bf370378e402756f54e848a0e09c7ab2f21d401a""",null,null,null,null,null,null,null,null,null,null,null,null,null
"""0x25d08081b4f88c61d4842ee9f6847cbe51f6a0eb171499e797c619fc8c197254""",null,null,null,null,null,null,null,null,null,null,null,null,null
"""0x6cab531c0475314c37629e22f091438ba299d646cc9137720be45299d3323afb""",null,null,null,null,null,null,null,null,null,null,null,null,null
"""0xeaf0a73e8065541d017297b83d2e9723c6d4ef1c86e1b9f8ebe643e1e75eaf96""",null,null,null,null,null,null,null,null,null,null,null,null,null


In [12]:
commitment_stored_replace_df = pl.read_parquet('data/commitment_store.parquet')

In [13]:
commitment_stored_replace_df

block_number,transaction_index,log_index,transaction_hash,address,topic0,n_data_bytes,chain_id,event__commitmentIndex,event__bidder,event__commiter,event__bid,event__blockNumber,event__bidHash,event__decayStartTimeStamp,event__decayEndTimeStamp,event__txnHash,event__commitmentHash,event__bidSignature,event__commitmentSignature,event__dispatchTimestamp,event__sharedSecretKey
u32,u32,u32,str,str,str,u32,u64,str,str,str,u64,u64,str,u64,u64,str,str,str,str,u64,str
7103000,0,0,"""0x37575bb9ae4781a43c9cd3a7a3e27d890701ad4dfe5a07c85983db2d6bc97cdc""","""0xcac68d97a56b19204dd3dbdc103cb24d47a825a3""","""0xa4aab50afc443b845214b8f4e2e7c32ea42be39a84e532be779802c54ff8ffda""",832,17864,"""0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000f9000000…","""0xe51ef1836dbef052bffd2eb3fe1314365d23129d""","""0xf21b2308b84a60aca2066dcec6e9b1378b45f922""",1000000000,1858548,"""0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000ea000000…",1720040096844,1720040112844,"""3c933d6510ec1d5c0897f0637f84b9b28872940b8904536333fb7aabe4602ba5""","""0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000e5000000…","""0x00000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000041000000000000000000000000000000000000000000000000000000000000002b000000…","""0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000004100000000000000000000000000000000000000000000000000000000000000a4000000…",1720040105015,"""0x00000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000009c000000…"
7084472,0,0,"""0x4a2d3f2f05a4d8b2233a58e96557cc6f5c62ed201d0b339ed911702862829bef""","""0xcac68d97a56b19204dd3dbdc103cb24d47a825a3""","""0xa4aab50afc443b845214b8f4e2e7c32ea42be39a84e532be779802c54ff8ffda""",832,17864,"""0x00000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000005d000000…","""0xe51ef1836dbef052bffd2eb3fe1314365d23129d""","""0xf21b2308b84a60aca2066dcec6e9b1378b45f922""",1000000000,1858270,"""0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000f0000000…",1720036456050,1720036472050,"""8f5c2721679eb319d8e988da7f2442ebf88ad1a0e1ff01e1f7c900e2cc33a95e""","""0x000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000046000000…","""0x000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000410000000000000000000000000000000000000000000000000000000000000090000000…","""0x000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000410000000000000000000000000000000000000000000000000000000000000092000000…",1720036464377,"""0x000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000037000000…"
6695070,0,0,"""0x825da19f1f6400b96345c63cb31375edb60f712620d25841ec488f3d2140aac2""","""0xcac68d97a56b19204dd3dbdc103cb24d47a825a3""","""0xa4aab50afc443b845214b8f4e2e7c3

In [14]:
# attempt to join on block_number and event_blockNumber
commitment_stored_replace_df.drop('block_number').rename(
    {'event__blockNumber': 'block_number'}).join(commitment_stored_logs_df, on='block_number')

transaction_index,log_index,transaction_hash,address,topic0,n_data_bytes,chain_id,event__commitmentIndex,event__bidder,event__commiter,event__bid,block_number,event__bidHash,event__decayStartTimeStamp,event__decayEndTimeStamp,event__txnHash,event__commitmentHash,event__bidSignature,event__commitmentSignature,event__dispatchTimestamp,event__sharedSecretKey,removed,log_index_right,transaction_index_right,transaction_hash_right,block_hash,address_right,data,topic0_right,topic1,topic2,topic3
u32,u32,str,str,str,u32,u64,str,str,str,u64,u64,str,u64,u64,str,str,str,str,u64,str,bool,u64,u64,str,str,str,str,str,str,str,str
